In [1]:
import pandas as pd
import json
import openai
import os
from openai import OpenAI
import time
from dotenv import load_dotenv

In [21]:
file_path = "/Users/priyal/Documents/honours/datasets/qa_dataset_normal-500.csv"  
df = pd.read_csv(file_path)

jsonl_path = "/Users/priyal/Documents/honours/datasets/qa_dataset_normal-500.jsonl"

with open(jsonl_path, "w") as f:
    for _, row in df.iterrows():
        jsonl_entry = {
            "messages": [
                {"role": "system", "content": "You are an expert in Ayurveda."},
                {"role": "user", "content": f"{row['instruction']}\n{row['input']}"},
                {"role": "assistant", "content": row["output"]},
            ]
        }
        f.write(json.dumps(jsonl_entry) + "\n")

print(f"Dataset converted and saved as {jsonl_path}")

Dataset converted and saved as /Users/priyal/Documents/honours/datasets/qa_dataset_normal-500.jsonl


In [2]:
load_dotenv("/Users/priyal/Documents/honours/.env/key.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
dataset_path = "/Users/priyal/Documents/honours/json/qa_dataset_normal-500.jsonl"

In [4]:
with open(dataset_path, "rb") as file:
    response = client.files.create(file=file, purpose="fine-tune")

file_id = response.id
print(f"File ID: {file_id}")

File ID: file-43qPZwge4KEGXyDbQW2gsn


In [5]:
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={"batch_size": 4, "n_epochs": 5},
)

fine_tune_id = fine_tune_response.id
print(f"Fine-tune ID: {fine_tune_id}")


Fine-tune ID: ftjob-fPniFzHxhbAAI4DkuRDQnz1K


In [6]:
while True:
    response = client.fine_tuning.jobs.retrieve(fine_tune_id)
    status = response.status

    print(f"Status: {status}")

    if status in ["succeeded", "failed", "cancelled"]:
        break

    time.sleep(60)

print(f"Fine-tuning job completed with status: {status}")

Status: validating_files
Status: validating_files
Status: validating_files
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: succeeded
Fine-tuning job completed with status: succeeded


In [37]:
prompt = """How can Ayurveda contribute to the management and treatment of epilepsy"""
context = """Epilepsy, a chronic non-communicable disease of the brain, is one of the most common neurological diseases globally that affects people of all ages. The existence of medical, neurological, psychiatric, and cognitive comorbidities has always undermined the available advanced treatment strategies for epilepsy. New-generation antiepileptic drugs being less successful in completely controlling the seizures and observance of complex diseases, including drug-resistant cases, have provided scope for integrating and incorporating the therapeutic modalities of Ayurveda, the ancient Indian art of holistic medicine, in the effective management of epilepsy. Epilepsy can be correlated to Apasmara, described in the classics of Ayurveda as the transient appearance of unconsciousness with loathsome expression due to derangement of memory, intelligence, and mind. The multifaceted therapeutic approach of Ayurveda, which involves pharmacologic and nonpharmacologic measures, purificatory and pacifying procedures, herbal and herbo-mineral formulations, disease, and host-specific approaches, have enhanced the potential of not only relieving symptoms but also modifying the pathophysiology of the disease. Newer paradigms of research in Ayurveda, along with holistic and integrative approaches with contemporary medicine, can not only benefit the existing healthcare system but also impact future healthcare management in epileptology research. This cursory literature review is an earnest attempt to identify, evaluate, and summarize various studies and provide a comprehensive insight into the potential of Ayurveda in understanding and treating epilepsy."""
input_text = f"Context: {context}\n\nPrompt: {prompt}"

In [ ]:
response = client.chat.completions.create(
    model=fine_tune_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": input_text},
    ],
    max_tokens=300,
)

print(response.choices[0].message.content.strip())

Ayurveda can contribute to the management and treatment of epilepsy in several ways through its holistic and multifaceted approach to health and wellness. Here are key aspects of how Ayurveda can be integrated into the treatment of epilepsy:

1. **Holistic Assessment**: Ayurveda emphasizes a comprehensive understanding of an individual’s unique constitution (Prakriti), imbalances (Vikriti), and the specific nature of their epilepsy. This personalized assessment allows for tailored treatment strategies that consider physical, mental, and emotional health.

2. **Pharmacological Interventions**: Ayurvedic medicine utilizes a range of herbal formulations that may help manage seizures and their symptoms. Herbs such as Brahmi (Bacopa monnieri), Ashwagandha (Withania somnifera), and Vacha (Acorus calamus) are traditionally thought to support brain health, improve cognition, and reduce anxiety, potentially offering complementary benefits in epilepsy management.

3. **Dietary Recommendations**: